# Fig.1 Artifact Reproduction: Bitmask vs Coordinate List

Reproduces the key results from micro22-sparseloop-artifact Fig.1 using AccelForge.

**Architecture:** BackingStorage (SRAM) → Buffer (SRAM) → Reg → MAC
**Workload:** SpMSpM Z[m,n] = A[m,k] * B[k,n], M=K=N=128
**Formats:** Bitmask (gating) vs Coordinate List / CSR (skipping)

In [ ]:
import os
import sys
import tempfile

import yaml
import pandas as pd

# Add accelforge to path
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, REPO_ROOT)

from accelforge.frontend.spec import Spec
from accelforge.model.main import evaluate_mapping

FIG1_DIR = os.path.join(REPO_ROOT, 'tests', 'input_files', 'fig1')
print(f'Using configs from: {FIG1_DIR}')

## 1. Configuration Files

AccelForge uses YAML configuration files for architecture, workload, mapping, and sparse optimizations.

In [ ]:
# Display architecture configuration
with open(os.path.join(FIG1_DIR, 'arch_unified.yaml')) as f:
    print('=== Architecture (unified) ===')
    print(f.read())

In [ ]:
# Display workload and mapping
for name in ['workload.yaml', 'mapping.yaml']:
    with open(os.path.join(FIG1_DIR, name)) as f:
        print(f'=== {name} ===')
        print(f.read())
        print()

In [ ]:
# Display sparse configurations
for name in ['sparse_bitmask_latency.yaml', 'sparse_coord_list_latency.yaml']:
    with open(os.path.join(FIG1_DIR, name)) as f:
        print(f'=== {name} ===')
        print(f.read())
        print()

## 2. Helper Functions

In [ ]:
def make_workload_yaml(density):
    """Generate workload dict with given density for A and B."""
    return {
        'workload': {
            'iteration_space_shape': {
                'm': '0 <= m < 128',
                'n': '0 <= n < 128',
                'k': '0 <= k < 128',
            },
            'bits_per_value': {'All': 8},
            'einsums': [{
                'name': 'SpMSpM',
                'tensor_accesses': [
                    {'name': 'A', 'projection': ['m', 'k'], 'density': density},
                    {'name': 'B', 'projection': ['n', 'k'], 'density': density},
                    {'name': 'Z', 'projection': ['m', 'n'], 'output': True},
                ],
            }],
        }
    }


def run_config(density, arch_yaml, sparse_yaml):
    """Run a single configuration and return (cycles, energy, result)."""
    workload = make_workload_yaml(density)
    with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
        yaml.dump(workload, f)
        wf = f.name
    try:
        spec = Spec.from_yaml(
            os.path.join(FIG1_DIR, arch_yaml),
            wf,
            os.path.join(FIG1_DIR, 'mapping.yaml'),
            os.path.join(FIG1_DIR, sparse_yaml),
        )
        result = evaluate_mapping(spec)
        cycles = float(result.data['Total<SEP>latency'].iloc[0])
        energy = float(result.data['Total<SEP>energy'].iloc[0])
        return cycles, energy, result
    finally:
        os.unlink(wf)


def get_component_latency(result, component):
    """Get per-component latency."""
    for col in result.data.columns:
        if col.endswith(f'latency<SEP>{component}'):
            return float(result.data[col].iloc[0])
    return 0.0

## 3. Dense Baseline (d=0.1015625)

In [ ]:
# Dense baseline (no sparse optimizations)
spec = Spec.from_yaml(
    os.path.join(FIG1_DIR, 'arch_latency.yaml'),
    os.path.join(FIG1_DIR, 'workload.yaml'),
    os.path.join(FIG1_DIR, 'mapping.yaml'),
)
dense_result = evaluate_mapping(spec)
dense_cycles = float(dense_result.data['Total<SEP>latency'].iloc[0])
dense_energy = float(dense_result.data['Total<SEP>energy'].iloc[0])

print(f'Dense baseline:')
print(f'  Total cycles: {dense_cycles:,.0f}')
print(f'  Total energy: {dense_energy:,.2f} pJ')
print()
for comp in ['BackingStorage', 'Buffer', 'Reg', 'MAC']:
    lat = get_component_latency(dense_result, comp)
    print(f'  {comp:>15}: {lat:>12,.0f} cycles')

## 4. Bitmask (Gating) at d=0.1015625

In [ ]:
bm_cycles, bm_energy, bm_result = run_config(
    0.1015625, 'arch_latency.yaml', 'sparse_bitmask_latency.yaml'
)
_, bm_energy_only, bm_energy_result = run_config(
    0.1015625, 'arch_energy.yaml', 'sparse_bitmask_energy.yaml'
)

print(f'Bitmask (gating) at d=0.1015625:')
print(f'  Total cycles: {bm_cycles:,.0f}')
print(f'  Total energy: {bm_energy_only:,.2f} pJ ({bm_energy_only/1e6:.4f} uJ)')
print()
for comp in ['BackingStorage', 'Buffer', 'Reg', 'MAC']:
    lat = get_component_latency(bm_result, comp)
    print(f'  {comp:>15}: {lat:>12,.0f} cycles')
print()
print(f'  Sparseloop reference: 2,113,536 cycles, ~2.27 uJ')

## 5. Coordinate List (Skipping) at d=0.1015625

In [ ]:
cl_cycles, cl_energy, cl_result = run_config(
    0.1015625, 'arch_latency.yaml', 'sparse_coord_list_latency.yaml'
)
_, cl_energy_only, cl_energy_result = run_config(
    0.1015625, 'arch_energy.yaml', 'sparse_coord_list_energy.yaml'
)

print(f'Coord list (skipping) at d=0.1015625:')
print(f'  Total cycles: {cl_cycles:,.0f}')
print(f'  Total energy: {cl_energy_only:,.2f} pJ ({cl_energy_only/1e6:.4f} uJ)')
print()
for comp in ['BackingStorage', 'Buffer', 'Reg', 'MAC']:
    lat = get_component_latency(cl_result, comp)
    print(f'  {comp:>15}: {lat:>12,.0f} cycles')
print()
print(f'  Sparseloop reference: 295,152 cycles, ~2.92 uJ')

## 6. Comparison Table: AccelForge vs Sparseloop

In [ ]:
speed_ratio = cl_cycles / bm_cycles

comparison = pd.DataFrame({
    'Metric': ['Bitmask cycles', 'Coord list cycles', 'Speed ratio (CL/BM)',
               'Bitmask energy (uJ)', 'Coord list energy (uJ)'],
    'AccelForge': [
        f'{bm_cycles:,.0f}',
        f'{cl_cycles:,.0f}',
        f'{speed_ratio:.4f}',
        f'{bm_energy_only/1e6:.4f}',
        f'{cl_energy_only/1e6:.4f}',
    ],
    'Sparseloop': [
        '2,113,536',
        '295,152',
        '0.1396',
        '2.27',
        '2.92',
    ],
})
display(comparison)

## 7. Density Sweep

In [ ]:
DENSITIES = [0.01, 0.02, 0.04, 0.08, 0.1, 0.2, 0.4, 0.8]

# Sparseloop ground truth
SL_BM_CYCLES = [2113536] * 8
SL_CL_CYCLES = [34056, 58124, 116247, 232490, 295152, 578952, 1157904, 3698200]
SL_BM_ENERGY = [1.34, 1.42, 1.62, 2.04, 2.27, 3.38, 5.93, 12.29]
SL_CL_ENERGY = [0.39, 0.62, 1.18, 2.31, 2.92, 5.77, 11.87, 25.41]

bm_cycles_sweep, cl_cycles_sweep = [], []
bm_energy_sweep, cl_energy_sweep = [], []

print(f'{"Density":>8} | {"BM cycles":>12} | {"CL cycles":>12} | '
      f'{"BM energy":>12} | {"CL energy":>12} | {"Speed":>8} | {"Energy":>8}')
print('-' * 90)

for d in DENSITIES:
    bm_c, _, _ = run_config(d, 'arch_latency.yaml', 'sparse_bitmask_latency.yaml')
    cl_c, _, _ = run_config(d, 'arch_latency.yaml', 'sparse_coord_list_latency.yaml')
    _, bm_e, _ = run_config(d, 'arch_energy.yaml', 'sparse_bitmask_energy.yaml')
    _, cl_e, _ = run_config(d, 'arch_energy.yaml', 'sparse_coord_list_energy.yaml')
    
    bm_cycles_sweep.append(bm_c)
    cl_cycles_sweep.append(cl_c)
    bm_energy_sweep.append(bm_e / 1e6)  # Convert to uJ
    cl_energy_sweep.append(cl_e / 1e6)
    
    sr = cl_c / bm_c
    er = cl_e / bm_e
    print(f'{d:8.2f} | {bm_c:12,.0f} | {cl_c:12,.0f} | '
          f'{bm_e/1e6:12.4f} | {cl_e/1e6:12.4f} | {sr:8.4f} | {er:8.4f}')

## 8. Plot: Normalized Speed Ratio vs Density

In [ ]:
import matplotlib.pyplot as plt

af_speed = [cl / bm for cl, bm in zip(cl_cycles_sweep, bm_cycles_sweep)]
sl_speed = [cl / bm for cl, bm in zip(SL_CL_CYCLES, SL_BM_CYCLES)]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(DENSITIES, af_speed, 'o-', label='AccelForge', color='tab:blue', linewidth=2)
ax.plot(DENSITIES, sl_speed, 's--', label='Sparseloop', color='tab:orange', linewidth=2)
ax.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5)
ax.set_xlabel('Density', fontsize=12)
ax.set_ylabel('Normalized Speed (CoordList / Bitmask)', fontsize=12)
ax.set_title('Fig.1a: Speed Ratio vs Density', fontsize=14)
ax.set_xscale('log')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Plot: Normalized Energy Ratio vs Density

In [ ]:
af_energy_ratio = [cl / bm for cl, bm in zip(cl_energy_sweep, bm_energy_sweep)]
sl_energy_ratio = [cl / bm for cl, bm in zip(SL_CL_ENERGY, SL_BM_ENERGY)]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(DENSITIES, af_energy_ratio, 'o-', label='AccelForge', color='tab:blue', linewidth=2)
ax.plot(DENSITIES, sl_energy_ratio, 's--', label='Sparseloop', color='tab:orange', linewidth=2)
ax.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5, label='Break-even')
ax.set_xlabel('Density', fontsize=12)
ax.set_ylabel('Normalized Energy (CoordList / Bitmask)', fontsize=12)
ax.set_title('Fig.1b: Energy Ratio vs Density', fontsize=14)
ax.set_xscale('log')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Analysis

### Key Findings

1. **Bitmask cycles are constant** at 2,113,536 across all densities (gating never saves cycles)
2. **Coord list cycles scale linearly** with density (skipping eliminates bandwidth)
3. **Speed ratio matches Sparseloop** closely after Phase 11 fixes (~0.14 at d=0.1)
4. **Energy crossover** at ~d=0.3: coord list cheaper below, bitmask cheaper above

### Remaining Differences

- AccelForge uses an analytical hypergeometric model vs Sparseloop's distribution-dependent simulation
- Absolute energy values may differ due to ERT calibration (trends match)
- Coord list cycle values are very close to Sparseloop ground truth